<h3> Benchmark Bert Model using GPU or CPU <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
import torch
from transformers import BertModel
import time
from datetime import timedelta

<b>2. Set the file location of Bert model<b>

In [2]:
modelPath = '/media/agemagician/Disk2/share_files/summit/uniref100/bert/30_layers/'

<b>3. Load Bert Model<b>

In [3]:
model = BertModel.from_pretrained(modelPath)

<b>4. Load the model into the GPU if avilabile and switch to inference mode<b>

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
model = model.to(device)
model = model.eval()

<b>5. Benchmark Configuration<b>

In [6]:
min_batch_size = 8
max_batch_size = 32
inc_batch_size = 8

min_sequence_length = 64
max_sequence_length = 512
inc_sequence_length = 64

iterations = 10

<b>6. Start Benchmarking<b>

In [7]:
device_name = torch.cuda.get_device_name(device.index) if device.type == 'cuda' else 'CPU'

with torch.no_grad():
    print((' Benchmarking using ' + device_name + ' ').center(80, '*'))
    print(' Start '.center(80, '*'))
    for sequence_length in range(min_sequence_length,max_sequence_length+1,inc_sequence_length):
        for batch_size in range(min_batch_size,max_batch_size+1,inc_batch_size):
            start = time.time()
            for i in range(iterations):
                input_ids = torch.randint(1, 20, (batch_size,sequence_length)).cuda()
                results = model(input_ids)[0].cpu().numpy()
            end = time.time()
            ms_per_protein = (end-start)/(iterations*batch_size)
            print('Sequence Length: %4d \t Batch Size: %4d \t Ms per protein %4.2f' %(sequence_length,batch_size,ms_per_protein))
        print(' Done '.center(80, '*'))
    print(' Finished '.center(80, '*'))

************************** Benchmarking using TITAN V **************************
************************************ Start *************************************
Sequence Length:   64 	 Batch Size:    8 	 Ms per protein 0.01
Sequence Length:   64 	 Batch Size:   16 	 Ms per protein 0.01
Sequence Length:   64 	 Batch Size:   24 	 Ms per protein 0.01
Sequence Length:   64 	 Batch Size:   32 	 Ms per protein 0.00
************************************* Done *************************************
Sequence Length:  128 	 Batch Size:    8 	 Ms per protein 0.01
Sequence Length:  128 	 Batch Size:   16 	 Ms per protein 0.01
Sequence Length:  128 	 Batch Size:   24 	 Ms per protein 0.01
Sequence Length:  128 	 Batch Size:   32 	 Ms per protein 0.01
************************************* Done *************************************
Sequence Length:  192 	 Batch Size:    8 	 Ms per protein 0.02
Sequence Length:  192 	 Batch Size:   16 	 Ms per protein 0.02
Sequence Length:  192 	 Batch Size:   24 	 Ms 